In [2]:
# Canadian Online Grocery Price Tracker - COMPLETE FIXED VERSION
# Enhanced with historical simulation and guaranteed Statistics Canada export

# INSTALL AND IMPORT REQUIRED LIBRARIES

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import random
from datetime import datetime, timedelta
import warnings
import json
import zipfile
import io
import os
import re
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# REAL PRODUCT URLS FOR CANADIAN RETAILERS

REAL_PRODUCT_URLS = {
    "Walmart": {
        "Milk 2% 4L": "https://www.walmart.ca/en/ip/Sealtest-Partly-Skimmed-2-Milk/6000199044832",
        "Large Eggs 18-pack": "https://www.walmart.ca/en/ip/Gray-Ridge-Premium-Large-White-Eggs/6000191268613",
        "yogurt 6% M.F 750g": "https://www.walmart.ca/en/ip/Astro-Original-Balkan-Style-Plain-Yogurt-6/10165982",
        "Coffee Cream 10% Half & Half 1L": "https://www.walmart.ca/en/ip/Great-Value-Half-Half-Cream/6000196187777",
        "whole chicken": "https://www.walmart.ca/en/ip/prime-whole-chicken-raised-without-antibiotics/6000199462076",
        "White Bread 675gm": "https://www.walmart.ca/en/ip/great-value-white-bread/6000206630542",
        "Long Grain Rice 900g": "https://www.walmart.ca/en/ip/Great-Value-Long-Grain-White-Rice/4UD9P76BEQB4",
        "Lean Ground Beef 450gm": "https://www.walmart.ca/en/ip/lean-ground-beef-your-fresh-market/6000191288138",
        "Bananas": "https://www.walmart.ca/en/ip/banana/875806",
        "Apples": "https://www.walmart.ca/en/ip/apple-royal-gala-your-fresh-market/6000197346282",
        "Potatoes": "https://www.walmart.ca/en/ip/your-fresh-market-russet-potatoes/6000196075655",
        "Cheddar Cheese": "https://www.walmart.ca/en/ip/great-value-old-cheddar-cheese/6000208884796"
    },
    "Metro": {
        "Milk 2% 4L": "https://www.instacart.ca/products/19461863-beatrice-2-milk-4-l?retailerSlug=metro",
        "Large Eggs 18-pack": "https://www.instacart.ca/products/27266594-selection-large-white-eggs-18-ct?retailerSlug=metro",
        "yogurt 6% M.F 750g": "https://www.instacart.ca/products/17874024-original-balkan-style-natural-yogurt-750-g?retailerSlug=metro",
        "Coffee Cream 10% Half & Half 1L": "https://www.instacart.ca/products/18454998-lactantia-10-half-half-cream-1-l?retailerSlug=metro",
        "whole chicken": "https://www.instacart.ca/products/27245877-zabiha-whole-chicken-per-kg?retailerSlug=metro",
        "White Bread 675gm": "https://www.instacart.ca/products/17818596-wonder-bread-free-white-bread-675-g?retailerSlug=metro",
        "Long Grain Rice 900g": "https://www.instacart.ca/products/27245111-selection-white-long-grain-rice-900-g?retailerSlug=metro",
        "Lean Ground Beef 450gm": "https://www.instacart.ca/products/30496898-naturalia-lean-ground-beef-454-g?retailerSlug=metro",
        "Bananas": "https://www.instacart.ca/products/2748189-banana-each?retailerSlug=metro",
        "Apples": "https://www.instacart.ca/products/16614400-honeycrisp-apple-each?retailerSlug=metro",
        "Potatoes": "https://www.instacart.ca/products/16417856-russet-potato-bag-5-lbs?retailerSlug=metro",
        "Cheddar Cheese": "https://www.instacart.ca/products/27268458-selection-old-cheddar-colored-cheese-400-g?retailerSlug=metro"
    },
    "Loblaws": {
        "Milk 2% 4L": "https://www.instacart.ca/products/17881458-2-trutaste-milk-4000-ml?retailerSlug=loblaws",
        "Large Eggs 18-pack": "https://www.instacart.ca/products/52088856-conestoga-farms-free-run-eggs-18-pack?retailerSlug=loblaws",
        "yogurt 6% M.F 750g": "https://www.instacart.ca/products/17874024-original-balkan-style-natural-yogurt-750-g?retailerSlug=loblaws",
        "Coffee Cream 10% Half & Half 1L": "https://www.instacart.ca/products/17818861-neilson-10-milk-fat-fresh-half-half-cream-carton-1000-ml?retailerSlug=loblaws",
        "whole chicken": "https://www.instacart.ca/products/75114607-air-chilled-whole-chicken?retailerSlug=loblaws",
        "White Bread 675gm": "https://www.instacart.ca/products/17818596-wonder-bread-free-white-bread-675-g?retailerSlug=loblaws",
        "Long Grain Rice 900g": "https://www.instacart.ca/products/21319248-dainty-foods-royal-white-grain-rice-each?retailerSlug=loblaws",
        "Lean Ground Beef 450gm": "https://www.instacart.ca/products/44685959-lean-ground-beef-450-g?retailerSlug=loblaws",
        "Bananas": "https://www.instacart.ca/products/17327587-banana-bunch-each?retailerSlug=loblaws",
        "Apples": "https://www.instacart.ca/products/20619816-president-s-choice-organic-gala-apples?retailerSlug=loblaws",
        "Potatoes": "https://www.instacart.ca/products/30665693-president-s-choice-russet-potatoes-2-27-kg?retailerSlug=loblaws",
        "Cheddar Cheese": "https://www.instacart.ca/products/21425157-no-name-old-yellow-cheddar-cheese-400-g?retailerSlug=loblaws"
    }
}

# STATISTICS CANADA API INTEGRATION

class StatisticsCanadaAPI:
    """Fetch real-time inflation data from Statistics Canada."""

    def __init__(self):
        self.base_url = "https://www150.statcan.gc.ca/t1/wds/rest"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (compatible; GroceryTracker/1.0)',
            'Accept': 'application/json, text/csv, */*'
        })

    def fetch_cpi_vectors(self):
        """Fetch Consumer Price Index data using vector approach."""
        try:
            print("Fetching Statistics Canada CPI data via vectors...")
            food_vector = "41690973"
            vector_url = f"{self.base_url}/getDataFromVectorsAndLatestNPeriods"
            post_data = [{"vectorId": int(food_vector), "latestN": 24}]

            response = self.session.post(vector_url, json=post_data, timeout=30)
            response.raise_for_status()
            data = response.json()

            if not data or len(data) == 0:
                raise Exception("No data returned from Statistics Canada API")

            records = []
            for item in data:
                if item.get('status') == 'SUCCESS' and 'object' in item:
                    obj = item['object']
                    if 'vectorDataPoint' in obj:
                        for point in obj['vectorDataPoint']:
                            records.append({
                                'REF_DATE': pd.to_datetime(point['refPer']),
                                'VALUE': float(point['value']) if point['value'] else None,
                                'Products and product groups': 'Food purchased from stores'
                            })

            if not records:
                raise Exception("No valid data points extracted")

            df = pd.DataFrame(records)
            df = df.dropna(subset=['VALUE'])
            df = df.sort_values('REF_DATE')
            df['inflation_yoy'] = df['VALUE'].pct_change(periods=12) * 100

            print(f"Retrieved {len(df)} Statistics Canada CPI records")
            return df

        except Exception as e:
            print(f"Failed to fetch Statistics Canada vector data: {e}")
            return self._fetch_fallback_data()

    def fetch_cpi_data(self, table_id="18-10-0004-01"):
        """Primary method - try vector approach first, fall back to table download."""
        vector_data = self.fetch_cpi_vectors()
        if len(vector_data) > 0:
            return vector_data
        return self._fetch_fallback_data()

    def _fetch_fallback_data(self):
        """Generate fallback data when API is unavailable."""
        print("Using fallback historical data...")

        statcan_food_inflation = {
            "2024-01": 3.4, "2024-02": 2.4, "2024-03": 1.9, "2024-04": 1.4,
            "2024-05": 1.5, "2024-06": 2.1, "2024-07": 2.1, "2024-08": 2.4,
            "2024-09": 2.4, "2024-10": 2.7, "2024-11": 2.6, "2024-12": 1.9,
            "2025-01": 1.9, "2025-02": 2.8, "2025-03": 3.2, "2025-04": 3.8,
            "2025-05": 3.3, "2025-06": 2.8, "2025-07": 3.4
        }

        fallback_data = []
        base_value = 100.0

        for date_str, inflation_rate in statcan_food_inflation.items():
            date = pd.to_datetime(date_str, format='%Y-%m')
            fallback_data.append({
                'REF_DATE': date,
                'Products and product groups': 'Food purchased from stores',
                'VALUE': base_value,
                'inflation_yoy': inflation_rate
            })
            base_value *= (1 + inflation_rate / 100 / 12)

        print(f"Using {len(fallback_data)} months of actual StatCan food inflation data")
        return pd.DataFrame(fallback_data)

# HISTORICAL PRICE SIMULATOR

class HistoricalPriceSimulator:
    """Generate historical prices based on Statistics Canada inflation data."""

    def __init__(self):
        # Current Walmart prices (scraped baseline from September 2025)
        self.current_baseline_prices = {
            'Milk 2% 4L': 6.25,
            'Large Eggs 18-pack': 6.88,
            'yogurt 6% M.F 750g': 3.48,
            'Coffee Cream 10% Half & Half 1L': 3.77,
            'whole chicken': 18.58,
            'White Bread 675gm': 1.97,
            'Long Grain Rice 900g': 3.27,
            'Lean Ground Beef 450gm': 8.46,
            'Bananas': 1.68,
            'Apples': 5.97,
            'Potatoes': 4.97,
            'Cheddar Cheese': 4.98
        }

        # Statistics Canada food inflation rates (year-over-year %)
        self.statcan_inflation_rates = {
            "2024-01": 3.4, "2024-02": 2.4, "2024-03": 1.9, "2024-04": 1.4,
            "2024-05": 1.5, "2024-06": 2.1, "2024-07": 2.1, "2024-08": 2.4,
            "2024-09": 2.4, "2024-10": 2.7, "2024-11": 2.6, "2024-12": 1.9,
            "2025-01": 1.9, "2025-02": 2.8, "2025-03": 3.2, "2025-04": 3.8,
            "2025-05": 3.3, "2025-06": 2.8, "2025-07": 3.4
        }

    def generate_historical_prices(self, months_back=24):
        """Generate historical prices for all products based on StatCan inflation."""
        historical_data = []
        current_date = datetime.now()

        for product_name, current_price in self.current_baseline_prices.items():
            product_prices = []
            working_price = current_price

            # Work backwards from current month
            for i in range(months_back):
                target_date = current_date - timedelta(days=30*i)
                year_month_key = target_date.strftime("%Y-%m")

                # Get inflation rate for that month
                if year_month_key in self.statcan_inflation_rates:
                    monthly_inflation = self.statcan_inflation_rates[year_month_key]
                    monthly_rate = monthly_inflation / 12 / 100
                else:
                    monthly_rate = 2.5 / 12 / 100  # Default average

                # Apply product-specific variation
                product_variation = self._get_product_inflation_multiplier(product_name)
                adjusted_monthly_rate = monthly_rate * product_variation

                # Add realistic noise
                noise = random.uniform(-0.1, 0.1) * abs(adjusted_monthly_rate)
                final_monthly_rate = adjusted_monthly_rate + noise

                # Calculate price for this month (working backwards)
                if i == 0:
                    monthly_price = current_price
                else:
                    working_price = working_price / (1 + final_monthly_rate)
                    monthly_price = working_price

                # Ensure realistic price bounds
                monthly_price = max(monthly_price, current_price * 0.5)

                # Round to realistic pricing
                if monthly_price > 10:
                    monthly_price = round(monthly_price, 2)
                else:
                    monthly_price = round(monthly_price * 4) / 4

                product_prices.append({
                    'product_name': product_name,
                    'store_name': 'Walmart',
                    'price': monthly_price,
                    'scraped_at': target_date,
                    'success': True,
                    'data_source': 'historical_simulation',
                    'scraping_method': 'statcan_inflation_based',
                    'year_month': target_date.strftime("%Y-%m"),
                    'inflation_rate_used': monthly_inflation if year_month_key in self.statcan_inflation_rates else 2.5
                })

            historical_data.extend(reversed(product_prices))

        return historical_data

    def _get_product_inflation_multiplier(self, product_name):
        """Product-specific inflation multipliers based on food category behavior."""
        multipliers = {
            # Dairy products - track close to average inflation
            'Milk 2% 4L': 1.0,
            'yogurt 6% M.F 750g': 0.9,
            'Coffee Cream 10% Half & Half 1L': 0.95,
            'Cheddar Cheese': 1.1,

            # Proteins - more volatile, higher inflation
            'Large Eggs 18-pack': 1.4,  # Very volatile
            'whole chicken': 1.2,
            'Lean Ground Beef 450gm': 1.3,

            # Staples - more stable pricing
            'White Bread 675gm': 0.8,
            'Long Grain Rice 900g': 0.9,

            # Fresh produce - most volatile
            'Bananas': 1.5,
            'Apples': 1.3,
            'Potatoes': 1.4
        }

        return multipliers.get(product_name, 1.0)

# ENHANCED WEB SCRAPING CLASS

class FixedGroceryPriceScraper:
    """Fixed web scraper that actually attempts to scrape Walmart prices online."""

    def __init__(self, delay_range=(5, 12)):
        """Initialize scraper with enhanced anti-detection measures."""
        self.session = requests.Session()
        self.delay_range = delay_range
        self.scraping_stats = {
            'live_scraped': 0,
            'fallback_used': 0,
            'failed_completely': 0
        }

        # Enhanced headers to avoid detection
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-CA,en-US;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0'
        }
        self.session.headers.update(self.headers)
        self.price_data = []

        # Historical baseline prices (only used as absolute last resort)
        self.historical_baseline_prices = {
            'milk': 6.25, 'eggs': 6.88, 'yogurt': 3.48, 'cream': 3.77, 'cheese': 4.98,
            'chicken': 18.58, 'beef': 8.46, 'bread': 1.97, 'rice': 3.27,
            'bananas': 1.68, 'apples': 5.97, 'potatoes': 4.97,
        }

    def create_walmart_session(self):
        """Create a specialized session for Walmart scraping."""
        try:
            print("  Initializing Walmart session...")
            home_response = self.session.get("https://www.walmart.ca", timeout=20)
            home_response.raise_for_status()

            self.session.headers.update({
                'Referer': 'https://www.walmart.ca/',
                'Origin': 'https://www.walmart.ca',
                'Host': 'www.walmart.ca'
            })

            time.sleep(random.uniform(3, 6))
            print("  Walmart session initialized successfully")
            return True

        except Exception as e:
            print(f"  Failed to initialize Walmart session: {e}")
            return False

    def attempt_walmart_scraping(self, url, product_name):
        """Enhanced Walmart-specific scraping with multiple strategies."""
        print(f"  Attempting to scrape Walmart price for {product_name}...")

        if not self.create_walmart_session():
            print("  Failed to create Walmart session, will use fallback")
            return None

        walmart_user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        ]

        for attempt, user_agent in enumerate(walmart_user_agents, 1):
            try:
                print(f"    Attempt {attempt}/{len(walmart_user_agents)} with different user agent")

                self.session.headers.update({'User-Agent': user_agent})

                if attempt > 1:
                    delay = random.uniform(8, 15)
                    print(f"    Waiting {delay:.1f} seconds before retry...")
                    time.sleep(delay)

                response = self.session.get(url, timeout=25)

                if response.status_code == 403:
                    print(f"    Attempt {attempt}: 403 Forbidden - Walmart blocking request")
                    continue
                elif response.status_code == 429:
                    print(f"    Attempt {attempt}: 429 Too Many Requests - Rate limited")
                    if attempt < len(walmart_user_agents):
                        time.sleep(random.uniform(15, 25))
                        continue
                elif response.status_code != 200:
                    print(f"    Attempt {attempt}: HTTP {response.status_code}")
                    continue

                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                price = self._extract_walmart_price(soup)

                if price and 0.50 < price < 500:
                    print(f"    SUCCESS: Found price ${price:.2f} on attempt {attempt}")
                    self.scraping_stats['live_scraped'] += 1
                    return price
                else:
                    print(f"    Attempt {attempt}: No valid price found in page content")

            except requests.RequestException as e:
                print(f"    Attempt {attempt} failed: {str(e)[:100]}...")
                continue
            except Exception as e:
                print(f"    Attempt {attempt} error: {str(e)[:100]}...")
                continue

        print("  All Walmart scraping attempts failed")
        return None

    def _extract_walmart_price(self, soup):
        """Enhanced price extraction specifically for Walmart.ca."""

        walmart_price_selectors = [
            '[data-automation-id="product-price"] span[itemprop="price"]',
            '[data-automation-id="product-price"] .price-display',
            '.price-display .visuallyhidden',
            '[data-testid="price-current"] span',
            '.price-current span',
            '.current-price',
            '.price-current',
            '[itemprop="price"]',
            '.price .visuallyhidden',
            '[data-automation-id="product-price"]',
            '.selling-price-list__item__price',
            '.price-block__highlight',
            '.price',
            '[class*="price"]'
        ]

        for selector in walmart_price_selectors:
            try:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    if text:
                        price = self.extract_price_from_text(text)
                        if price and 0.50 < price < 500:
                            return price
            except:
                continue

        # Try searching in script tags for JSON data
        try:
            script_tags = soup.find_all('script', type='application/ld+json')
            for script in script_tags:
                try:
                    data = json.loads(script.string)
                    if isinstance(data, dict):
                        price = self._extract_price_from_json(data)
                        if price and 0.50 < price < 500:
                            return price
                except:
                    continue
        except:
            pass

        return None

    def _extract_price_from_json(self, data):
        """Extract price from JSON structured data."""
        if isinstance(data, dict):
            price_fields = ['price', 'lowPrice', 'highPrice', 'offers']

            for field in price_fields:
                if field in data:
                    if isinstance(data[field], (int, float)):
                        return float(data[field])
                    elif isinstance(data[field], str):
                        price = self.extract_price_from_text(data[field])
                        if price:
                            return price
                    elif isinstance(data[field], dict) and 'price' in data[field]:
                        return float(data[field]['price'])

            for value in data.values():
                if isinstance(value, dict):
                    price = self._extract_price_from_json(value)
                    if price:
                        return price
                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            price = self._extract_price_from_json(item)
                            if price:
                                return price
        return None

    def extract_price_from_text(self, text):
        """Enhanced price extraction with better regex patterns."""
        if not text:
            return None

        text = re.sub(r'\s+', ' ', text.strip())

        price_patterns = [
            r'\$(\d+(?:\.\d{2})?)',
            r'CAD\s*(\d+(?:\.\d{2})?)',
            r'C\$\s*(\d+(?:\.\d{2})?)',
            r'(\d+)\.(\d{2})',
            r'(\d+)\s*\.\s*(\d{2})',
            r'(\d+)\s+(\d{2})',
            r'Price:\s*\$?(\d+(?:\.\d{2})?)',
            r'Current price\s*\$?(\d+(?:\.\d{2})?)',
        ]

        for pattern in price_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                try:
                    match = matches[0]
                    if isinstance(match, tuple):
                        if len(match) == 2:
                            dollars, cents = match
                            return float(f"{dollars}.{cents}")
                    else:
                        return float(match)
                except (ValueError, IndexError):
                    continue
        return None

    def attempt_generic_scraping(self, url, product_name, store_name):
        """Generic scraping for non-Walmart retailers."""
        print(f"  Attempting to scrape {store_name} price for {product_name}...")

        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        ]

        for attempt, user_agent in enumerate(user_agents, 1):
            try:
                print(f"    Attempt {attempt}/{len(user_agents)}")

                self.session.headers.update({'User-Agent': user_agent})

                if attempt > 1:
                    delay = random.uniform(10, 18)
                    print(f"    Waiting {delay:.1f} seconds...")
                    time.sleep(delay)

                response = self.session.get(url, timeout=25)

                if response.status_code == 429:
                    print(f"    Attempt {attempt}: Rate limited (429)")
                    if attempt < len(user_agents):
                        time.sleep(random.uniform(20, 30))
                        continue
                elif response.status_code == 403:
                    print(f"    Attempt {attempt}: Access forbidden (403)")
                    continue
                elif response.status_code != 200:
                    print(f"    Attempt {attempt}: HTTP {response.status_code}")
                    continue

                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                price = self._extract_generic_price(soup)

                if price and 0.50 < price < 500:
                    print(f"    SUCCESS: Found price ${price:.2f}")
                    self.scraping_stats['live_scraped'] += 1
                    return price
                else:
                    print(f"    Attempt {attempt}: No valid price found")

            except requests.RequestException as e:
                print(f"    Attempt {attempt} failed: {str(e)[:80]}...")
                continue
            except Exception as e:
                print(f"    Attempt {attempt} error: {str(e)[:80]}...")
                continue

        print(f"  All {store_name} scraping attempts failed")
        return None

    def _extract_generic_price(self, soup):
        """Generic price extraction for Metro/Loblaws (Instacart)."""

        generic_price_selectors = [
            '[data-testid="price"]',
            '.css-price span',
            '.price-display',
            '[class*="price"] span',
            '.price', '.price-current', '.current-price',
            '.selling-price', '.regular-price', '.sale-price',
            '[data-automation-id*="price"]',
            '[itemprop="price"]',
            '.product-price',
            '[class*="Price"]'
        ]

        for selector in generic_price_selectors:
            try:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text(strip=True)
                    if text:
                        price = self.extract_price_from_text(text)
                        if price and 0.50 < price < 500:
                            return price
            except:
                continue
        return None

    def generate_market_based_price(self, product_name, store_name):
        """Generate realistic market-based prices only as fallback when scraping fails."""

        store_multipliers = {
            'Walmart': 1.0,
            'Metro': 1.15,
            'Loblaws': 1.18,
        }

        product_lower = product_name.lower()
        base_price = 5.00

        for key, price in self.historical_baseline_prices.items():
            if key in product_lower:
                base_price = price
                break

        store_multiplier = store_multipliers.get(store_name, 1.1)
        adjusted_price = base_price * store_multiplier

        variation = random.uniform(-0.04, 0.04)
        final_price = adjusted_price * (1 + variation)

        if final_price > 10:
            final_price = round(final_price, 2)
        else:
            final_price = round(final_price * 4) / 4

        return final_price

    def scrape_product_price(self, url, product_name, store_name):
        """Main scraping method - tries real scraping first, fallback only if needed."""
        print(f"Scraping {product_name} from {store_name}...")

        scraped_price = None
        data_source = 'failed_scraping'

        try:
            if 'walmart.ca' in url.lower():
                scraped_price = self.attempt_walmart_scraping(url, product_name)
            else:
                scraped_price = self.attempt_generic_scraping(url, product_name, store_name)

            if scraped_price and 0.50 < scraped_price < 500:
                print(f"  SUCCESS: Live scraped price ${scraped_price:.2f}")
                data_source = 'live_scraped'

                return {
                    'product_name': product_name,
                    'store_name': store_name,
                    'price': scraped_price,
                    'url': url,
                    'scraped_at': datetime.now(),
                    'success': True,
                    'data_source': data_source,
                    'scraping_method': 'live_scraping'
                }

            else:
                print("  Live scraping failed, generating market-based price...")
                fallback_price = self.generate_market_based_price(product_name, store_name)
                data_source = 'market_simulation'
                self.scraping_stats['fallback_used'] += 1

                print(f"  Using market-based price: ${fallback_price:.2f}")

                return {
                    'product_name': product_name,
                    'store_name': store_name,
                    'price': fallback_price,
                    'url': url,
                    'scraped_at': datetime.now(),
                    'success': True,
                    'data_source': data_source,
                    'scraping_method': 'fallback_simulation',
                    'note': 'Generated price due to scraping failure'
                }

        except Exception as e:
            print(f"  Unexpected error: {str(e)[:100]}...")
            fallback_price = self.generate_market_based_price(product_name, store_name)
            self.scraping_stats['failed_completely'] += 1

            print(f"  Using emergency fallback price: ${fallback_price:.2f}")

            return {
                'product_name': product_name,
                'store_name': store_name,
                'price': fallback_price,
                'url': url,
                'scraped_at': datetime.now(),
                'success': True,
                'data_source': 'emergency_fallback',
                'scraping_method': 'emergency_fallback',
                'error': str(e)[:200],
                'note': 'Emergency fallback due to system error'
            }

    def scrape_all_products(self, product_urls):
        """Scrape all products with detailed progress tracking."""
        print("Starting enhanced grocery price scraping...")
        print("=" * 60)

        all_results = []
        total_products = sum(len(products) for products in product_urls.values())
        current_count = 0

        for store_name, products in product_urls.items():
            print(f"\n--- Scraping {store_name} ---")
            self.session.close()
            self.session = requests.Session()
            self.session.headers.update(self.headers)

            for product_name, url in products.items():
                current_count += 1
                print(f"Progress: {current_count}/{total_products}")

                result = self.scrape_product_price(url, product_name, store_name)
                all_results.append(result)
                self.price_data.append(result)
                delay = random.uniform(*self.delay_range)
                if current_count < total_products:
                    print(f"  Waiting {delay:.1f}s before next product...")
                    time.sleep(delay)

        # Print detailed statistics
        success_count = sum(1 for r in all_results if r['success'])
        live_scraped = sum(1 for r in all_results if r.get('data_source') == 'live_scraped')
        market_simulation = sum(1 for r in all_results if r.get('data_source') == 'market_simulation')
        emergency_fallback = sum(1 for r in all_results if r.get('data_source') == 'emergency_fallback')

        print(f"\n" + "=" * 60)
        print("SCRAPING RESULTS SUMMARY")
        print("=" * 60)
        print(f"Total products processed: {len(all_results)}")
        print(f"Successfully processed: {success_count}")
        print(f"Live scraped prices: {live_scraped} ({live_scraped/len(all_results)*100:.1f}%)")
        print(f"Market-based simulations: {market_simulation} ({market_simulation/len(all_results)*100:.1f}%)")
        print(f"Emergency fallbacks: {emergency_fallback}")

        if live_scraped > 0:
            print(f"\nSUCCESS: {live_scraped} prices were actually scraped from websites!")
        else:
            print(f"\nWARNING: No prices were successfully scraped from websites")
            print("   All prices are market-based simulations")

        # Show examples of successfully scraped prices
        live_scraped_results = [r for r in all_results if r.get('data_source') == 'live_scraped']
        if live_scraped_results:
            print(f"\nSuccessfully scraped prices:")
            for result in live_scraped_results[:5]:  # Show first 5
                print(f"  {result['store_name']} - {result['product_name']}: ${result['price']:.2f}")

        return all_results

# DATA STORAGE AND MANAGEMENT

class DynamicDataManager:
    """Manage persistent storage of price data with historical tracking."""

    def __init__(self, data_file='data/historical_prices.csv'):
        self.data_file = data_file
        os.makedirs(os.path.dirname(data_file), exist_ok=True)

    def load_historical_data(self):
        """Load existing historical price data."""
        try:
            if os.path.exists(self.data_file):
                df = pd.read_csv(self.data_file)
                df['scraped_at'] = pd.to_datetime(df['scraped_at'])
                print(f"Loaded {len(df)} historical price records")
                return df
            else:
                print("No historical data found, starting fresh")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error loading historical data: {e}")
            return pd.DataFrame()

    def append_new_data(self, new_data):
        """Append new price data to historical records."""
        if not new_data:
            print("No new data to append")
            return pd.DataFrame()

        # Load existing data
        historical_df = self.load_historical_data()

        # Convert new data to DataFrame
        new_df = pd.DataFrame(new_data)
        new_df = new_df[new_df['success'] == True]  # Only successful scrapes

        if len(new_df) == 0:
            print("No successful scrapes to save")
            return pd.DataFrame()

        # Combine and deduplicate
        if len(historical_df) > 0:
            combined_df = pd.concat([historical_df, new_df], ignore_index=True)
        else:
            combined_df = new_df

        # Remove duplicates based on product, store, and date
        combined_df['scraped_date'] = pd.to_datetime(combined_df['scraped_at']).dt.date
        combined_df = combined_df.drop_duplicates(
            subset=['product_name', 'store_name', 'scraped_date'],
            keep='last'
        )

        # Save updated data
        combined_df.to_csv(self.data_file, index=False)
        print(f"Saved {len(combined_df)} total price records")

        return combined_df

# ENHANCED INFLATION ANALYSIS

class DynamicInflationAnalyzer:
    """Enhanced inflation analyzer using real price data and Statistics Canada integration."""

    def __init__(self, price_data, statcan_api=None):
        self.price_data = price_data.copy() if len(price_data) > 0 else pd.DataFrame()
        self.statcan_api = statcan_api or StatisticsCanadaAPI()
        self.official_cpi_data = None

    def prepare_price_data(self):
        """Prepare price data for analysis."""
        if len(self.price_data) == 0:
            print("No price data available for analysis")
            return pd.DataFrame()

        df = self.price_data.copy()
        df['scraped_at'] = pd.to_datetime(df['scraped_at'])
        df['date'] = df['scraped_at'].dt.date
        df['year_month'] = df['scraped_at'].dt.to_period('M')

        # Create monthly averages
        monthly_data = df.groupby([
            'product_name', 'store_name', 'year_month'
        ]).agg({
            'price': 'mean',
            'scraped_at': 'count'
        }).round(2)

        monthly_data.columns = ['avg_price', 'obs_count']
        monthly_data = monthly_data.reset_index()

        print(f"Prepared {len(monthly_data)} monthly price records for analysis")
        return monthly_data

    def compare_with_official_cpi(self):
        """Compare grocery tracker results with official Statistics Canada CPI."""
        print("Comparing with Statistics Canada CPI data...")

        # Fetch official CPI data and store it for export
        self.official_cpi_data = self.statcan_api.fetch_cpi_data()

        if len(self.official_cpi_data) == 0:
            print("No official CPI data available for comparison")
            return pd.DataFrame()  # Return empty DataFrame instead of None

        # Prepare our data
        monthly_data = self.prepare_price_data()
        if len(monthly_data) == 0:
            return pd.DataFrame()

        # Calculate our basket inflation
        basket_inflation = monthly_data.groupby('year_month').agg({
            'avg_price': 'mean'
        }).reset_index()

        basket_inflation = basket_inflation.sort_values('year_month')
        basket_inflation['our_inflation_yoy'] = basket_inflation['avg_price'].pct_change(periods=12) * 100

        # Merge with official data
        basket_inflation['date'] = basket_inflation['year_month'].dt.to_timestamp()
        official_monthly = self.official_cpi_data.groupby(
            self.official_cpi_data['REF_DATE'].dt.to_period('M')
        )['inflation_yoy'].mean().reset_index()

        official_monthly.columns = ['year_month', 'official_inflation_yoy']

        comparison = basket_inflation.merge(
            official_monthly, on='year_month', how='inner'
        )

        if len(comparison) > 0:
            comparison['difference'] = comparison['our_inflation_yoy'] - comparison['official_inflation_yoy']

            print("\nComparison with Statistics Canada:")
            print("-" * 60)
            for _, row in comparison.tail(6).iterrows():
                print(f"{row['year_month']}: Our {row['our_inflation_yoy']:5.1f}% vs Official {row['official_inflation_yoy']:5.1f}% (diff: {row['difference']:+5.1f}%)")

        return comparison

    def generate_dynamic_report(self):
        """Generate comprehensive report with real data."""
        print("\nGenerating dynamic inflation analysis report...")

        monthly_data = self.prepare_price_data()
        comparison_data = self.compare_with_official_cpi()

        # Count data sources
        live_scraped_count = len(self.price_data[self.price_data.get('data_source', '') == 'live_scraped'])
        market_sim_count = len(self.price_data[self.price_data.get('data_source', '') == 'market_simulation'])
        historical_sim_count = len(self.price_data[self.price_data.get('data_source', '') == 'historical_simulation'])
        emergency_count = len(self.price_data[self.price_data.get('data_source', '') == 'emergency_fallback'])

        report = {
            'analysis_date': datetime.now(),
            'data_summary': {
                'total_price_points': len(self.price_data),
                'date_range': {
                    'start': self.price_data['scraped_at'].min() if len(self.price_data) > 0 else None,
                    'end': self.price_data['scraped_at'].max() if len(self.price_data) > 0 else None
                },
                'products_tracked': len(self.price_data['product_name'].unique()) if len(self.price_data) > 0 else 0,
                'stores_covered': len(self.price_data['store_name'].unique()) if len(self.price_data) > 0 else 0,
                'live_scraped_prices': live_scraped_count,
                'market_simulated_prices': market_sim_count,
                'historical_simulated_prices': historical_sim_count,
                'emergency_fallback_prices': emergency_count,
                'scraping_success_rate': round(live_scraped_count / len(self.price_data) * 100, 1) if len(self.price_data) > 0 else 0
            },
            'comparison_available': comparison_data is not None and len(comparison_data) > 0
        }

        if comparison_data is not None and len(comparison_data) > 0:
            report['latest_comparison'] = {
                'our_inflation': comparison_data['our_inflation_yoy'].iloc[-1],
                'official_inflation': comparison_data['official_inflation_yoy'].iloc[-1],
                'difference': comparison_data['difference'].iloc[-1]
            }

        return report

# MAIN EXECUTION - COMPLETE FIXED VERSION

def run_complete_grocery_tracker():
    """Execute the complete grocery price tracker with guaranteed Statistics Canada export."""
    print("=" * 80)
    print("COMPLETE CANADIAN GROCERY PRICE TRACKER - WITH GUARANTEED STATCAN EXPORT")
    print("=" * 80)

    # Ensure required directories exist
    os.makedirs('data', exist_ok=True)
    os.makedirs('powerbi_export', exist_ok=True)

    # Initialize components
    scraper = FixedGroceryPriceScraper()
    data_manager = DynamicDataManager()
    statcan_api = StatisticsCanadaAPI()
    historical_simulator = HistoricalPriceSimulator()

    # Load historical data
    historical_data = data_manager.load_historical_data()

    # Generate historical prices based on Statistics Canada inflation
    print("\nGenerating historical prices based on Statistics Canada inflation data...")
    simulated_historical_data = historical_simulator.generate_historical_prices(months_back=24)
    print(f"Generated {len(simulated_historical_data)} historical price points")

    # Scrape current prices
    print("\nStarting comprehensive price scraping...")
    print("This version will attempt to scrape ALL prices online first")
    print("Fallback prices only used when scraping fails")

    new_results = scraper.scrape_all_products(REAL_PRODUCT_URLS)

    # Combine current scraped data with historical simulated data
    print("\nCombining current prices with historical simulated data...")
    combined_results = simulated_historical_data + new_results
    print(f"Total data points: {len(combined_results)} (Historical: {len(simulated_historical_data)}, Current: {len(new_results)})")

    # Save combined data
    print("\nSaving combined price data...")
    updated_data = data_manager.append_new_data(combined_results)

    if updated_data is not None and len(updated_data) > 0:
        # Perform analysis
        print("\nAnalyzing price trends with historical context...")
        analyzer = DynamicInflationAnalyzer(updated_data, statcan_api)
        report = analyzer.generate_dynamic_report()

        # GUARANTEED STATISTICS CANADA EXPORT
        print("\nExporting comprehensive data for PowerBI...")
        monthly_data = analyzer.prepare_price_data()

        # Always export basic files
        if len(monthly_data) > 0:
            # 1. Export monthly data
            monthly_data['year_month_str'] = monthly_data['year_month'].astype(str)
            monthly_data.to_csv('powerbi_export/monthly_prices.csv', index=False)

            # 2. Export current prices
            current_prices = updated_data.groupby(['product_name', 'store_name']).tail(1)
            current_prices.to_csv('powerbi_export/current_prices.csv', index=False)

            # 3. Export price history
            price_history = updated_data[['product_name', 'store_name', 'price', 'scraped_at', 'data_source']].copy()
            price_history['date'] = pd.to_datetime(price_history['scraped_at']).dt.date
            price_history.to_csv('powerbi_export/price_history.csv', index=False)

            # 4. Export scraping report
            scraping_report = updated_data[['store_name', 'data_source', 'scraping_method']].groupby(['store_name', 'data_source', 'scraping_method']).size().reset_index(name='count')
            scraping_report.to_csv('powerbi_export/scraping_success_report.csv', index=False)

        # GUARANTEED STATISTICS CANADA EXPORTS
        print("Exporting Statistics Canada data (guaranteed)...")

        # 5. Always export Statistics Canada CPI data
        if hasattr(analyzer, 'official_cpi_data') and analyzer.official_cpi_data is not None and len(analyzer.official_cpi_data) > 0:
            statcan_export = analyzer.official_cpi_data.copy()
            statcan_export['year'] = statcan_export['REF_DATE'].dt.year
            statcan_export['month'] = statcan_export['REF_DATE'].dt.month
            statcan_export['year_month'] = statcan_export['REF_DATE'].dt.to_period('M').astype(str)
            statcan_export.to_csv('powerbi_export/statistics_canada_cpi.csv', index=False)
            print("Statistics Canada CPI data exported successfully")
        else:
            print("No Statistics Canada data available")

        # 6. Always export grocery basket trends
        if len(monthly_data) > 0:
            basket_trends = monthly_data.groupby('year_month').agg({
                'avg_price': ['mean', 'median', 'std', 'count']
            }).round(3)
            basket_trends.columns = ['basket_avg_price', 'basket_median_price', 'basket_price_std', 'product_count']
            basket_trends = basket_trends.reset_index()
            basket_trends = basket_trends.sort_values('year_month')

            # Calculate inflation rates
            basket_trends['our_inflation_mom'] = basket_trends['basket_avg_price'].pct_change() * 100
            basket_trends['our_inflation_yoy'] = basket_trends['basket_avg_price'].pct_change(periods=12) * 100
            basket_trends['year_month_str'] = basket_trends['year_month'].astype(str)
            basket_trends['date'] = basket_trends['year_month'].dt.to_timestamp()
            basket_trends['year'] = basket_trends['date'].dt.year
            basket_trends['month'] = basket_trends['date'].dt.month
            basket_trends.to_csv('powerbi_export/grocery_basket_inflation_trends.csv', index=False)
            print("Grocery basket inflation trends exported")

        # 7. Export comparison data (if available)
        comparison_data = analyzer.compare_with_official_cpi()
        if len(comparison_data) > 0:
            comparison_export = comparison_data.copy()
            comparison_export['year_month_str'] = comparison_export['year_month'].astype(str)
            comparison_export['date'] = comparison_export['year_month'].dt.to_timestamp()
            comparison_export['year'] = comparison_export['date'].dt.year
            comparison_export['month'] = comparison_export['date'].dt.month
            comparison_export.to_csv('powerbi_export/tracker_vs_statcan_comparison.csv', index=False)
            print("Tracker vs Statistics Canada comparison exported")
        else:
            print("No comparison data available (date mismatch)")

        # 8. Export product-level inflation analysis
        if len(monthly_data) > 0:
            product_inflation = []
            for product in updated_data['product_name'].unique():
                product_data = monthly_data[monthly_data['product_name'] == product].copy()
                if len(product_data) > 0:
                    product_data = product_data.sort_values('year_month')
                    product_data['price_change_mom'] = product_data['avg_price'].pct_change() * 100
                    product_data['price_change_yoy'] = product_data['avg_price'].pct_change(periods=12) * 100

                    for _, row in product_data.iterrows():
                        product_inflation.append({
                            'product_name': product,
                            'year_month': str(row['year_month']),
                            'date': row['year_month'].to_timestamp(),
                            'avg_price': row['avg_price'],
                            'inflation_mom': row['price_change_mom'],
                            'inflation_yoy': row['price_change_yoy'],
                            'store_count': row['obs_count']
                        })

            if product_inflation:
                product_inflation_df = pd.DataFrame(product_inflation)
                product_inflation_df['year'] = product_inflation_df['date'].dt.year
                product_inflation_df['month'] = product_inflation_df['date'].dt.month
                product_inflation_df.to_csv('powerbi_export/product_level_inflation.csv', index=False)
                print("Product-level inflation analysis exported")

        # Final summary
        print(f"\n" + "=" * 70)
        print("POWERBI FILES CREATED:")
        print("=" * 70)
        csv_files = [f for f in os.listdir('powerbi_export') if f.endswith('.csv')]
        for i, file in enumerate(sorted(csv_files), 1):
            file_path = f"powerbi_export/{file}"
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                print(f"{i:2d}. {file} ({len(df)} rows)")

        # Display comprehensive summary
        print(f"\n" + "=" * 70)
        print("ENHANCED TRACKER ANALYSIS SUMMARY")
        print("=" * 70)
        print(f"Total data points: {report['data_summary']['total_price_points']}")
        print(f"Historical simulated points: {report['data_summary']['historical_simulated_prices']}")
        print(f"Current scraped points: {len(new_results)}")
        print(f"Products tracked: {report['data_summary']['products_tracked']}")
        print(f"Stores covered: {report['data_summary']['stores_covered']}")
        print(f"Live scraped prices: {report['data_summary']['live_scraped_prices']} ({report['data_summary']['scraping_success_rate']}%)")
        print(f"Market-simulated prices: {report['data_summary']['market_simulated_prices']}")

        # Show data source breakdown including historical simulation
        print(f"\nData Source Breakdown:")
        source_breakdown = updated_data.groupby(['data_source']).size().reset_index(name='count')
        for _, row in source_breakdown.iterrows():
            print(f"  {row['data_source']}: {row['count']} data points")

        # Show historical vs current pricing examples
        print(f"\nHistorical vs Current Price Examples:")
        for product in ['Milk 2% 4L', 'Large Eggs 18-pack', 'whole chicken']:
            current_data = updated_data[
                (updated_data['product_name'] == product) &
                (updated_data['data_source'].isin(['live_scraped', 'market_simulation']))
            ]
            historical_data = updated_data[
                (updated_data['product_name'] == product) &
                (updated_data['data_source'] == 'historical_simulation')
            ]

            if len(current_data) > 0 and len(historical_data) > 0:
                current_price = current_data['price'].iloc[0]
                historical_price = historical_data['price'].iloc[0]
                price_change = ((current_price - historical_price) / historical_price * 100)
                print(f"  {product}: ${historical_price:.2f} → ${current_price:.2f} ({price_change:+.1f}%)")

        print(f"\n" + "=" * 70)
        print("SUCCESS: Complete tracker with guaranteed Statistics Canada export!")
        print("Real scraping attempts made")
        print("24 months of historical context based on Statistics Canada inflation")
        print("Comprehensive CSV files for PowerBI analysis")
        print("Statistics Canada CPI data guaranteed to export")

    else:
        print("No valid price data collected. Check URLs and network connection.")

# Run the complete tracker
if __name__ == "__main__":
    run_complete_grocery_tracker()

All libraries imported successfully!
Current date: 2025-09-25 12:51:12
COMPLETE CANADIAN GROCERY PRICE TRACKER - WITH GUARANTEED STATCAN EXPORT
Loaded 312 historical price records

Generating historical prices based on Statistics Canada inflation data...
Generated 288 historical price points

Starting comprehensive price scraping...
This version will attempt to scrape ALL prices online first
Fallback prices only used when scraping fails
Starting enhanced grocery price scraping...

--- Scraping Walmart ---
Progress: 1/36
Scraping Milk 2% 4L from Walmart...
  Attempting to scrape Walmart price for Milk 2% 4L...
  Initializing Walmart session...
  Walmart session initialized successfully
    Attempt 1/3 with different user agent
    Attempt 1: No valid price found in page content
    Attempt 2/3 with different user agent
    Waiting 11.7 seconds before retry...
    SUCCESS: Found price $6.25 on attempt 2
  SUCCESS: Live scraped price $6.25
  Waiting 10.3s before next product...
Progress: 